# X3FL dynamic relationships

## The relationship

In [1]:
import numpy as np
import pandas as pd

In [2]:
all_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran", "Yaki"]
# https://imgur.com/nqI0nbO, by blazenclaw, forum.egosoft.com
relationship_df = pd.read_csv("relationship.csv", index_col=0)
relationship_df = pd.DataFrame(relationship_df, columns=all_races)
relationship_df

,Goner,Teladi,NMMC,Strong Arms,TerraCorp,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,Yaki
Goner,1.00,0.00,0.00,0.00,0.00,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
Teladi,0.00,1.00,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-0.3
NMMC,0.00,0.15,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.30,0.00,0.00,0.0,0.0
Strong Arms,0.00,0.00,0.00,1.00,-0.15,-0.15,-0.30,0.15,-0.30,-0.15,0.00,0.05,0.00,-0.3,0.0
TerraCorp,0.00,0.00,0.00,-0.15,1.00,0.15,0.05,-0.15,0.05,-0.30,0.00,-0.30,-0.30,0.0,-0.3
Argon,0.05,0.00,0.00,-0.15,0.15,1.00,0.15,-0.15,0.05,0.15,-0.30,-0.30,-0.30,-0.3,-0.3
Boron,0.05,0.00,0.00,-0.30,0.05,0.15,1.00,-0.30,0.15,0.05,-0.30,-0.15,-0.30,-0.3,-0.3
Split,0.00,0.00,0.00,0.15,-0.15,-0.15,-0.30,1.00,-0.30,0.15,-0.30,0.15,0.00,-0.3,-0.3
Arteus,0.00,0.00,0.00,-0.30,0.05,0.05,0.15,-0.30,1.00,-0.30,0.00,-0.15,-0.30,-0.3,-0.3
OTAS,0.00,0.00,0.00,-0.15,-0.30,0.15,0.05,0.15,-0.30,1.00,0.00,-0.30,-0.30,-0.3,-0.3


The problem is simple: given the relationship $\mathbf{R}$, target notority $\mathbf{N}$, find tactics $\mathbf{X}$, which is a vector that represents how much notority point one should get from a race.

$$
\mathbf{R}\mathbf{X}=\mathbf{N}
$$

$$
\mathbf{X}=\mathbf{R}^{-1}\mathbf{N}
$$

Let's first define $\mathbf{N}$ as vectors of 1, since we don't want to lose effort by hitting the notoriety ceiling.

In [3]:
def get_X(races):
    R = np.array(relationship_df.loc[races, races])
    invR = np.linalg.inv(R) 
    N = np.ones(len(races))
    X = invR@N
    return X

def get_X_df(races):
    X = get_X(races)
    return pd.DataFrame(list(X)+[sum(X)]+[len(races)/sum(X)], index=races+['sum', 'efficiency']).transpose()

## Example solutions

Let's see if there is a solution for all the races

The number for each race represents how much effort you will have to put with that race to get 1 notoriety point for each race assuming you're to be friend with all these selected race.

The sum is the total workload of current tactic.

The efficiency is the sum of actual gained notoriety points (equal the number of races selected) divided by total workload.

In [4]:
get_X_df(all_races)

,Goner,Teladi,NMMC,Strong Arms,TerraCorp,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,Yaki,sum,efficiency
0,1.071172,0.415461,0.807539,-0.302713,-0.796793,-0.741422,-0.68202,-0.320156,-1.119438,-1.375322,-0.433805,-1.661266,-1.767547,-1.521107,-1.544692,-9.972109,-1.504195


Unfortunately there is not a solution as one would have to get negative points from races. The problem is, when actively losing points of a race, the notoriety points of enemy races does not increase accordingly. However if that is true, we can see a path that you do some missions for some races but keep robbing other races and one day you will find yourself master of diplomacy with all races - too good to be true for those races you robbed.

Let's get results of some interesting races combo.

In [5]:
commonwealth_races = ["Teladi", "Argon", "Boron", "Split", "Paranid"]
major_races = ["Goner", "Teladi", "NMMC", "Argon", "Boron", "Split", "Paranid", "Terran"]
all_but_yaki_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran"]
all_but_yaki_terran_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates"]
all_but_yaki_terran_pirates_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid"]
all_but_yaki_pirates_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Terran"]
all_but_terran_pirates_races = ["Goner", "Teladi", "NMMC", "Strong Arms", "TerraCorp", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Yaki"]

All commonwealth races, no problem

In [6]:
get_X_df(commonwealth_races)

,Teladi,Argon,Boron,Split,Paranid,sum,efficiency
0,1.0,1.428571,1.428571,1.428571,1.428571,6.714286,0.744681


All major races, no problem

In [7]:
get_X_df(major_races)

,Goner,Teladi,NMMC,Argon,Boron,Split,Paranid,Terran,sum,efficiency
0,0.623203,0.869565,0.869565,3.767968,3.767968,3.787443,3.787443,5.533247,23.006403,0.347729


All but Yaki, not an option

In [8]:
get_X_df(all_but_yaki_races)

,Goner,Teladi,NMMC,Strong Arms,TerraCorp,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,sum,efficiency
0,1.10937,0.904362,0.637588,-1.091562,-0.909377,-1.094362,-1.093043,-0.413423,-1.679534,-1.918018,-0.755858,-2.2777,-2.425656,-2.824747,-13.83196,-1.012149


Later we will learn that one of the best options, that if we want to have good relationship with as many races as possible, is as following

In [9]:
get_X_df(all_but_yaki_terran_pirates_races)

,Goner,Teladi,NMMC,Strong Arms,TerraCorp,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,sum,efficiency
0,0.219569,0.254951,4.966994,7.478574,7.592467,7.11746,8.491154,7.915631,7.876733,7.886442,13.350788,12.678057,85.828819,0.139813


Before that, let's consider another problem - the infinite notoriety we can get from Goner as a peaceful race. With Goner, the notoriety with Goner, Argon, Boron is actually infinite, thus not limited by 1. We should remove their possible influence, that is, to remove Goner, Argon, Boron.

In [10]:
infinite_races = ["Goner", "Argon", "Boron"]
all_races_without_infinite = ["Teladi", "NMMC", "Strong Arms", "TerraCorp", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran", "Yaki"]
major_races_without_infinite = ["Teladi", "NMMC", "Split", "Paranid", "Terran"]
all_but_yaki_races_without_infinite = ["Teladi", "NMMC", "Strong Arms", "TerraCorp", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran"]
all_but_yaki_terran_races_without_infinite = ["Teladi", "NMMC", "Strong Arms", "TerraCorp", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates"]
all_but_yaki_terran_pirates_races_without_infinite = ["Teladi", "NMMC", "Strong Arms", "TerraCorp", "Split", "Arteus", "OTAS", "Duke's", "Paranid"]
all_but_terran_pirates_races_without_infinite = ["Teladi", "NMMC", "Strong Arms", "TerraCorp", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Yaki"]

redo all above gives us the following, which does not change too much, however. Note the efficiency in the following chart is higher than actual, due to that Goner workload is not included yet. This will be addressed later.

In [11]:
get_X_df(all_races_without_infinite)

,Teladi,NMMC,Strong Arms,TerraCorp,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,Yaki,sum,efficiency
0,0.328493,0.959057,-0.422597,-1.436036,-0.008105,-1.681945,-2.245461,0.02777,-1.979967,-2.200839,-1.553343,-1.758827,-11.971799,-1.002356


In [12]:
get_X_df(all_but_yaki_races_without_infinite)

,Teladi,NMMC,Strong Arms,TerraCorp,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,sum,efficiency
0,0.87254,0.849735,-1.465397,-1.992687,0.090597,-2.746575,-3.486809,-0.064615,-2.959362,-3.299816,-3.179593,-17.381983,-0.632839


In [13]:
get_X_df(all_but_yaki_terran_races_without_infinite)

,Teladi,NMMC,Strong Arms,TerraCorp,Split,Arteus,OTAS,Duke's,Paranid,Pirates,sum,efficiency
0,0.88775,0.748335,-6.288639,-12.933661,0.967362,-11.554157,-16.267633,-0.395007,-14.058134,-15.384824,-74.278608,-0.134628


In [14]:
get_X_df(all_but_yaki_terran_pirates_races_without_infinite)

,Teladi,NMMC,Strong Arms,TerraCorp,Split,Arteus,OTAS,Duke's,Paranid,sum,efficiency
0,0.682387,2.11742,3.632319,5.212411,2.221933,5.208282,6.101887,4.065927,5.880404,35.122969,0.256243


In [15]:
get_X_df(all_but_terran_pirates_races_without_infinite)

,Teladi,NMMC,Strong Arms,TerraCorp,Split,Arteus,OTAS,Duke's,Paranid,Yaki,sum,efficiency
0,-1.184655,0.118922,-1.233279,-4.783234,-2.462803,-4.493523,-5.424689,-3.529255,-5.530914,-7.222722,-35.746151,-0.27975


## Let's explore all possibilities

In [16]:
# https://stackoverflow.com/questions/26332412/python-recursive-function-to-display-all-subsets-of-given-set
def subs(l):
    if l == []:
        return [[]]

    x = subs(l[1:])

    return x + [[l[0]] + y for y in x]

### The Goner approach

Turned out there are the following number of possible combos.

In [17]:
least_enemy_set_list_without_infinite = []
for races in subs(all_races_without_infinite):
    if races != []:
        if min(get_X(races))>0:
            enemy_set = set(all_races_without_infinite) - set(races)
            is_duplicate = False
            for i, least_enemy_set in enumerate(least_enemy_set_list_without_infinite):
                if enemy_set.issubset(least_enemy_set):
                    least_enemy_set_list_without_infinite[i] = enemy_set
                    is_duplicate = True
                    break
                if least_enemy_set.issubset(enemy_set):
                    is_duplicate = True
                    break
            if not is_duplicate:
                least_enemy_set_list_without_infinite = least_enemy_set_list_without_infinite + [enemy_set]
                
len(least_enemy_set_list_without_infinite)

824

which are parsed into races, workload, and efficiency. Note that to exploit Goner relationship bonus we can not gain points from argon and boron directly, and the 0.05 factor from Goner adds a lot of workloads.

In [18]:
parsed_least_enemy_set_list_without_infinite = []

r1 = np.array(relationship_df.loc[[infinite_races[0]], [infinite_races[0]]])[0,0]
r2 = np.array(relationship_df.loc[[infinite_races[0]], [infinite_races[1]]])[0,0]
r3 = np.array(relationship_df.loc[[infinite_races[0]], [infinite_races[2]]])[0,0]

for least_enemy_set in least_enemy_set_list_without_infinite:
    friend_races = list(set(all_races_without_infinite) - least_enemy_set)
    X = get_X(friend_races)
    workload_without_infinite = sum(X)
    R_extra = np.array(relationship_df.loc[infinite_races, friend_races])
    N_extra = R_extra@X
    workload_extra = max((1-N_extra[0])/r1,(1-N_extra[1])/r2,(1-N_extra[2])/r3)
    efficiency = (len(all_races) - len(list(least_enemy_set)))/(workload_without_infinite+workload_extra)
    item = " - ".join(list(least_enemy_set)) + ", " + str(workload_without_infinite+workload_extra) + ", " + str(efficiency)
    print(item)
    parsed_least_enemy_set_list_without_infinite += [item]

Split - OTAS - Arteus - TerraCorp, 1057.6083691205047, 0.010400825410588871
Yaki - Strong Arms - Arteus - Teladi - TerraCorp, 1410.8633621392462, 0.00708785858953579
Arteus - Terran - Teladi - TerraCorp, 1454.2439908459778, 0.007564067700634587
Pirates - Split - Arteus - Strong Arms - Teladi - TerraCorp, 2431.643270024765, 0.003701200793284263
Yaki - Pirates - Split - Strong Arms - TerraCorp, 396.11594526349927, 0.02524513370283018
Terran - TerraCorp - Yaki, 171.2032461286091, 0.07009212892485411
Arteus - Paranid - TerraCorp, 2023.7956074370447, 0.005929452537549937
Paranid - Yaki - Strong Arms - Teladi - TerraCorp, 11081.001168964041, 0.0009024455324495645
Paranid - Strong Arms - Terran - Teladi - TerraCorp, 11081.001168964132, 0.000902445532449557
Split - TerraCorp - Pirates - Terran, 4497.323613667418, 0.0024458991491230196
Arteus - Paranid - Duke's - TerraCorp, 276.50026848816077, 0.03978296317810266
Arteus - Duke's - TerraCorp - Yaki, 433.6020668910826, 0.02536888276126025
Arteus 

Paranid - TerraCorp - OTAS - Yaki, 437.0729480473949, 0.025167423536830727
Terran - Paranid - TerraCorp - OTAS, 288.82423959462847, 0.038085446067264835
Terran - OTAS - Pirates - TerraCorp, 384.77912262198214, 0.028587829622987913
Paranid - Pirates - Duke's - Split - TerraCorp, 335.5696488335274, 0.02980007290516579
Paranid - Yaki - Pirates - Duke's - TerraCorp, 148.8053764915649, 0.06720187291463124
Paranid - Duke's - Split - Terran - Teladi - TerraCorp, 375.2241379310344, 0.02398566374121215
Pirates - Duke's - Split - Terran - TerraCorp, 177.53868485308624, 0.05632575237489806
Terran - Paranid - Pirates - TerraCorp, 255.93690461756006, 0.042979342961254524
Arteus - Paranid - TerraCorp - OTAS, 307.33477120557416, 0.0357915895973976
Arteus - OTAS - TerraCorp - Yaki, 225.75422249399557, 0.04872555595407554
OTAS - Duke's - Arteus - Terran - TerraCorp, 98.77126758911992, 0.10124401806403002
OTAS - Pirates - Duke's - Arteus - TerraCorp, 95.3407653621142, 0.10488692808389942
Yaki - OTAS - P

Paranid - Pirates - Duke's - Strong Arms - Arteus - Terran, 70.09483457664747, 0.1283974782786406
Paranid - Yaki - Pirates - Duke's - Strong Arms - Arteus, 47.347185140806445, 0.19008521780618587
Paranid - Yaki - Duke's - Strong Arms - Arteus - Terran, 46.49012645071886, 0.1935894928042476
Yaki - Pirates - Duke's - Strong Arms - Arteus - Terran, 35.8779479155024, 0.2508504672897196
Paranid - Yaki - Pirates - Strong Arms - Arteus - Terran, 47.1932797702725, 0.19070511826705436
Paranid - OTAS - Duke's - Strong Arms - Terran - Teladi, 226.64643057897686, 0.03970942748583845
Paranid - Yaki - OTAS - Duke's - Strong Arms, 122.14133360357195, 0.08187236625773632
Paranid - Yaki - OTAS - Duke's - Strong Arms - Terran, 46.24383186776549, 0.19462055016841062
Yaki - OTAS - Pirates - Duke's - Strong Arms - Terran, 34.972961280553754, 0.2573416625432954
Paranid - Yaki - OTAS - Pirates - Strong Arms - Terran, 50.433850407671116, 0.17845157423536867
Paranid - Yaki - Pirates - Duke's - Strong Arms - Te

Yaki - Pirates - Duke's - Split - Strong Arms - NMMC, 160.01325861963363, 0.05624533915276256
Yaki - Duke's - Split - Terran - Arteus - NMMC, 187.3615023474178, 0.048035481607697716
Paranid - Pirates - Split - Terran - NMMC, 186.38955173147238, 0.05365107597021745
Paranid - Yaki - Pirates - Split - NMMC, 184.78323182848453, 0.054117464561297325
Paranid - Yaki - Split - Terran - Strong Arms - NMMC, 46.76748410535876, 0.1924413975257812
Yaki - Pirates - Split - Terran - NMMC, 78.4811892945176, 0.1274190680581157
Paranid - OTAS - Duke's - Split - NMMC, 1527.8716152889212, 0.006545052542329609
Yaki - OTAS - Duke's - Split - Strong Arms - NMMC, 295.6539898340373, 0.0304409894994215
Paranid - OTAS - Split - Terran - NMMC, 165.33512904453286, 0.06048321405009161
OTAS - Pirates - Duke's - Split - NMMC, 279.33969326475244, 0.03579870759907438
Yaki - OTAS - Pirates - Split - NMMC, 175.0970282737627, 0.0571111919978738
Yaki - OTAS - Split - Terran - NMMC, 93.4990897337801, 0.10695291289437142
Par

Pirates - Duke's - Split - Terran - Strong Arms - TerraCorp, 115.78775942641647, 0.07772842349298184
Paranid - Pirates - Strong Arms - Terran - TerraCorp, 204.44834169589498, 0.048912111084150625
Paranid - OTAS - Strong Arms - Arteus - TerraCorp, 278.2196692472231, 0.03594282182513165
Yaki - OTAS - Duke's - Strong Arms - Arteus - TerraCorp, 84.35525430023908, 0.10669163497471067
OTAS - Duke's - Strong Arms - Terran - Arteus - TerraCorp, 97.64100494767396, 0.09217438928269041
OTAS - Pirates - Duke's - Strong Arms - Arteus - TerraCorp, 83.3053984012848, 0.10803621581216992
Yaki - OTAS - Pirates - Strong Arms - Arteus - TerraCorp, 129.0676962569305, 0.06973084870194024
Yaki - OTAS - Strong Arms - Terran - Arteus - TerraCorp, 65.0170961736176, 0.13842513015295177
Paranid - OTAS - Pirates - Strong Arms - Arteus - TerraCorp, 189.30948635654386, 0.04754119919299489
Paranid - Yaki - OTAS - Strong Arms - Arteus - TerraCorp, 99.32982631544047, 0.09060722578351052
Paranid - OTAS - Strong Arms - T

with least enemies, which is 3, the results are

In [19]:
parsed_least_enemy_set_list_without_infinite = []
for least_enemy_set in least_enemy_set_list_without_infinite:
    if len(list(least_enemy_set)) == 3:
        friend_races = list(set(all_races_without_infinite) - least_enemy_set)
        X = get_X(friend_races)
        workload_without_infinite = sum(X)
        R_extra = np.array(relationship_df.loc[infinite_races, friend_races])
        N_extra = R_extra@X
        workload_extra = max((1-N_extra[0])/r1,(1-N_extra[1])/r2,(1-N_extra[2])/r3)
        efficiency = (len(all_races) - len(list(least_enemy_set)))/(workload_without_infinite+workload_extra)
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload_without_infinite+workload_extra) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list_without_infinite += [item]

Terran - TerraCorp - Yaki, 171.2032461286091, 0.07009212892485411
Arteus - Paranid - TerraCorp, 2023.7956074370447, 0.005929452537549937
Terran - Paranid - Yaki, 116.89992094755894, 0.10265190859609884
Terran - Pirates - Yaki, 105.3461102176374, 0.11391023337462458
Terran - OTAS - Yaki, 123.03150825209372, 0.0975359903368151
Paranid - OTAS - Yaki, 816.7694861889692, 0.014692027803330131
Arteus - OTAS - Yaki, 535.0506351864673, 0.022427783859779837
Arteus - Paranid - OTAS, 1324.7692323975446, 0.009058181384755296
Arteus - Paranid - Yaki, 297.80939829876394, 0.04029422868636784
Arteus - Paranid - Terran, 3824.2459278403912, 0.003137873511909988
Arteus - Paranid - Pirates, 2026.1585597183964, 0.005922537474889333


The same has been done with normal approach, that is to ignore Goner's peaceful nature and get Argon and Boron points directly.

### The normal approach

Turned out there are the following number of possible combos.

In [20]:
least_enemy_set_list = []
for races in subs(all_races):
    if races != []:
        if min(get_X(races))>0:
            enemy_set = set(all_races) - set(races)
            is_duplicate = False
            for i, least_enemy_set in enumerate(least_enemy_set_list):
                if enemy_set.issubset(least_enemy_set):
                    least_enemy_set_list[i] = enemy_set
                    is_duplicate = True
                    break
                if least_enemy_set.issubset(enemy_set):
                    is_duplicate = True
                    break
            if not is_duplicate:
                least_enemy_set_list = least_enemy_set_list + [enemy_set]
                
len(least_enemy_set_list)

4744

which is the following

In [21]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    workload = sum(get_X(list(set(all_races) - least_enemy_set)))
    efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
    item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
    print(item)
    parsed_least_enemy_set_list += [item]

OTAS - Boron - Split - Arteus - TerraCorp - Argon, 166.54797513966005, 0.054038483460714444
Yaki - Boron - Strong Arms - Arteus - Teladi - TerraCorp - Argon, 278.3952388716777, 0.028736123622026044
Boron - Arteus - Terran - Teladi - TerraCorp - Argon, 288.4090947263057, 0.03120567334584513
Pirates - Boron - Split - Arteus - Strong Arms - Teladi - TerraCorp - Argon, 476.3674649050357, 0.014694538388333168
Yaki - Pirates - Boron - Strong Arms - Split - TerraCorp - Argon, 100.16244463951448, 0.07987025505210132
Yaki - Strong Arms - Terran - Goner - Teladi - TerraCorp, 772.5786876184618, 0.01164929882772621
Paranid - Boron - Arteus - TerraCorp - Argon, 362.2912386041893, 0.027602102768279218
Paranid - Yaki - Boron - Strong Arms - Teladi - TerraCorp - Argon, 2838.3315961229455, 0.0028185572154175715
Paranid - Boron - Strong Arms - Terran - Teladi - TerraCorp - Argon, 2838.3315961229687, 0.002818557215417548
Pirates - Boron - Split - Terran - TerraCorp - Argon, 1257.4655385968397, 0.00715725

Yaki - Pirates - Duke's - Arteus - Terran, 17.90242128022806, 0.5585836599122087
Paranid - Yaki - Pirates - Arteus - Terran, 22.17108147532488, 0.4510379888833757
Paranid - OTAS - Pirates - Duke's - Strong Arms - Goner - Teladi - TerraCorp, 2639.11823236757, 0.0026524010611378543
Paranid - Yaki - OTAS - Duke's - Strong Arms - Argon, 108.99962884236942, 0.08256908849676373
Paranid - OTAS - Duke's - Terran - Goner - Teladi - TerraCorp - Argon, 128.78624768659324, 0.054353629566371055
OTAS - Pirates - Duke's - Terran - Goner - Teladi, 1782.6872916492916, 0.005048557894679025
Paranid - Yaki - OTAS - Pirates - Terran, 26.667041069698108, 0.3749947350312911
Paranid - Yaki - Pirates - Duke's - Terran, 16.962700637456543, 0.5895287674840108
Paranid - OTAS - Boron - Strong Arms - Split - Arteus - Goner, 468.8548267521469, 0.01706285089441786
Yaki - OTAS - Boron - Duke's - Strong Arms - Arteus - Goner, 531.0935398867756, 0.015063259857586534
OTAS - Boron - Duke's - Arteus - Terran - Teladi - Ter

Paranid - Pirates - Duke's - Terran - Teladi - Argon, 176.48392294856654, 0.050996146559043276
Paranid - Yaki - Pirates - Terran - Argon, 25.23893963889831, 0.39621315883603836
Paranid - OTAS - Duke's - Split - Arteus - Goner - Teladi - Argon, 176.4202354322888, 0.039677988088201166
Yaki - OTAS - Duke's - Strong Arms - Split - Arteus - Argon, 63.916982887228826, 0.12516235339385004
Paranid - OTAS - Split - Terran - Arteus - Goner - Teladi, 359.5080482992453, 0.022252631165967682
OTAS - Pirates - Duke's - Split - Arteus - Argon, 67.66890966764709, 0.13300051743412314
Yaki - OTAS - Pirates - Split - Arteus - Goner - Teladi - Argon, 169.6109685724232, 0.041270915784028604
Yaki - OTAS - Split - Terran - Argon, 46.10307835862345, 0.21690525570142385
Paranid - Pirates - Strong Arms - Split - Arteus - Argon, 110.39916476121888, 0.0815223558934164
Paranid - Yaki - Strong Arms - Split - Arteus - Argon, 58.18060997802723, 0.1546907123077428
Paranid - Yaki - Split - Terran - Argon, 46.66196162024

Paranid - Yaki - OTAS - Pirates - Boron - Goner - TerraCorp - NMMC, 580.9691504432174, 0.012048832532088405
Paranid - Yaki - OTAS - Terran - TerraCorp, 37.72309143055006, 0.26508962072767706
Yaki - OTAS - Pirates - Terran - TerraCorp, 45.865216548750816, 0.21803014904270326
Paranid - Pirates - Duke's - Terran - TerraCorp, 70.91294860763149, 0.1410179691628818
Paranid - Yaki - Duke's - Terran - TerraCorp, 29.256218930488135, 0.3418076691236037
Yaki - Pirates - Duke's - Terran - TerraCorp, 18.436161459910483, 0.5424122598267023
Paranid - Yaki - Pirates - Terran - TerraCorp, 25.698876921044207, 0.3891220628326849
Paranid - OTAS - Duke's - Split - Arteus - TerraCorp - Argon, 51.79329933780341, 0.15446013484915952
Yaki - OTAS - Duke's - Arteus - Goner - TerraCorp - Argon, 193.77657826554616, 0.04128465922768549
OTAS - Duke's - Split - Terran - Arteus - TerraCorp - Argon, 42.82964485251255, 0.18678651264909307
Pirates - Duke's - Split - Arteus - Goner - Teladi - TerraCorp - Argon, 402.622215

Yaki - OTAS - Pirates - Boron - Terran - Argon, 17.610075901795117, 0.5110710510386027
Paranid - Pirates - Boron - Duke's - Terran - Teladi - Argon, 63.76668147742279, 0.12545736761967888
Paranid - Yaki - Pirates - Boron - Duke's - Argon, 69.32495077033002, 0.129823388260549
Paranid - Yaki - Boron - Duke's - Terran - Argon, 38.79883251983331, 0.2319657426650493
Yaki - Pirates - Boron - Duke's - Terran - Argon, 24.734434547314976, 0.3638652010735773
Paranid - Yaki - Pirates - Boron - Terran - Argon, 17.769997914162936, 0.5064716407663096
Paranid - OTAS - Pirates - Boron - Duke's - Argon, 66.18071825652618, 0.13599127112997897
Paranid - Yaki - OTAS - Boron - Arteus - Argon, 23.075892611734165, 0.3900174156393617
Paranid - OTAS - Boron - Arteus - Terran - Argon, 37.073273128138226, 0.24276248738256387
OTAS - Pirates - Boron - Duke's - Arteus - Terran - Argon, 18.308262113185833, 0.4369611900103999
Yaki - OTAS - Pirates - Boron - Duke's - Arteus - Argon, 12.887402229835251, 0.6207612564058

...

OTAS - Pirates - Duke's - Strong Arms - Split - Terran - Teladi, 29.673169072169454, 0.26960382898580326
Paranid - Yaki - OTAS - Duke's - Strong Arms - Split - Teladi - TerraCorp, 26.758295995873496, 0.26160111245796436
Yaki - OTAS - Duke's - Strong Arms - Split - Terran - Teladi, 29.673169072169454, 0.26960382898580326
Yaki - OTAS - Pirates - Duke's - Strong Arms - Split - Terran - Teladi, 8.37860433862869, 0.8354613390355745
Paranid - Yaki - OTAS - Pirates - Strong Arms - Split - Terran - Teladi, 8.867319597420126, 0.789415552591179
Paranid - Yaki - Pirates - Duke's - Strong Arms - Split - Terran - Teladi, 7.3097243590126935, 0.9576284489262838
Paranid - OTAS - Pirates - Duke's - Strong Arms - Arteus - Teladi - TerraCorp, 36.4434739879659, 0.19207828546508735
Paranid - Yaki - OTAS - Pirates - Duke's - Strong Arms - Arteus - Teladi, 13.815843234388954, 0.5066646951071601
Yaki - OTAS - Duke's - Strong Arms - Arteus - Terran - Teladi, 25.94572230331382, 0.3083359910538405
Yaki - OTAS - 

Yaki - Pirates - Strong Arms - Split - Terran - NMMC, 25.98610308778363, 0.3463389631603134
OTAS - Pirates - Duke's - Strong Arms - Split - TerraCorp - Argon - NMMC, 32.67178942262025, 0.2142521154704053
Paranid - Yaki - OTAS - Strong Arms - Split - TerraCorp - Argon - NMMC, 23.87846532963451, 0.2931511679401192
Paranid - OTAS - Strong Arms - Split - Terran - Argon - NMMC, 79.03798666574907, 0.10121715313716083
OTAS - Pirates - Strong Arms - Split - Terran - TerraCorp - Argon - NMMC, 51.506744365145394, 0.13590453223708115
Yaki - OTAS - Pirates - Duke's - Strong Arms - Split - NMMC, 20.16666401423425, 0.39669426705147437
Yaki - OTAS - Strong Arms - Split - Terran - NMMC, 58.88395763909993, 0.15284298747650496
Paranid - OTAS - Pirates - Strong Arms - Split - Terran - NMMC, 28.325099421660752, 0.2824350192353516
Paranid - Yaki - OTAS - Pirates - Strong Arms - Split - NMMC, 18.154370037403293, 0.4406652493872092
Paranid - Yaki - OTAS - Strong Arms - Split - Terran - NMMC, 12.1925291831563

Yaki - OTAS - Pirates - Boron - Strong Arms - Split - Terran - NMMC, 12.968291580688348, 0.5397781162187938
Paranid - Pirates - Boron - Duke's - Strong Arms - Split - Terran - NMMC, 24.51204026010337, 0.285573943487415
Paranid - Yaki - Pirates - Boron - Duke's - Strong Arms - Split - NMMC, 13.374189928587885, 0.5233961860401884
Paranid - Yaki - Boron - Duke's - Strong Arms - Split - Terran - NMMC, 18.67374744656187, 0.37485780612765063
Yaki - Pirates - Boron - Duke's - Strong Arms - Split - Terran - NMMC, 15.178122017978737, 0.46119012561029515
Paranid - Yaki - Pirates - Boron - Strong Arms - Split - Terran - NMMC, 8.419628759159142, 0.8313905755506352
Paranid - OTAS - Pirates - Boron - Duke's - Strong Arms - Arteus - NMMC, 22.813203218763622, 0.3068398564144895
Paranid - Yaki - OTAS - Boron - Duke's - Strong Arms - Arteus - NMMC, 18.732764572210172, 0.37367682559702986
Paranid - OTAS - Boron - Duke's - Strong Arms - Arteus - Terran - NMMC, 41.21246827507494, 0.16985151079227057
OTAS -

OTAS - Pirates - Boron - Duke's - Split - Arteus - Argon - NMMC, 20.21893449158379, 0.34621013302722636
Paranid - Yaki - OTAS - Boron - Split - Arteus - Argon - NMMC, 12.51871110230645, 0.5591629955187893
Paranid - OTAS - Boron - Split - Arteus - Terran - Argon - NMMC, 16.918384676489858, 0.4137510840338881
Pirates - Boron - Duke's - Split - Arteus - Terran - Argon - NMMC, 88.67247954969882, 0.07894219306314386
Yaki - Pirates - Boron - Duke's - Split - Arteus - Argon - NMMC, 34.24029478228608, 0.20443749227361763
Yaki - OTAS - Boron - Split - Terran - Argon - NMMC, 21.55086497277996, 0.3712147985755784
Paranid - OTAS - Pirates - Boron - Split - Arteus - Argon - NMMC, 15.237437173289072, 0.4593948391971625
Paranid - Yaki - Pirates - Boron - Split - Arteus - Argon - NMMC, 15.837626559511483, 0.4419854183135833
Paranid - Yaki - Boron - Split - Arteus - Terran - Argon - NMMC, 13.194155509084489, 0.5305379336464796
Yaki - Pirates - Boron - Split - Terran - Argon - NMMC, 29.121471232713755, 

Yaki - OTAS - Pirates - Strong Arms - Split - Arteus - Terran - TerraCorp, 16.74801484665907, 0.41795998296457054
Paranid - Yaki - Duke's - Strong Arms - Split - Arteus - Terran - TerraCorp, 8.7733513517238, 0.797870701784288
Paranid - OTAS - Pirates - Duke's - Strong Arms - Split - Terran - TerraCorp, 10.669847137670452, 0.6560543848173921
Paranid - Yaki - OTAS - Duke's - Strong Arms - Split - Terran - TerraCorp, 8.773351351723802, 0.7978707017842878
Paranid - Yaki - OTAS - Duke's - Strong Arms - Arteus - Terran - TerraCorp, 9.818770641119608, 0.7129202072085278
Paranid - OTAS - Pirates - Boron - Strong Arms - Split - Arteus - Argon, 13.902698282910876, 0.5034993824619184
Paranid - Yaki - OTAS - Boron - Strong Arms - Split - Arteus - Argon, 10.423405304316955, 0.6715655580524037
Paranid - OTAS - Boron - Strong Arms - Split - Arteus - Terran - Argon, 16.363662240015167, 0.4277771013191917
Pirates - Boron - Duke's - Split - Strong Arms - Arteus - Terran - Argon, 63.15756244853142, 0.110

Note that the despite total number gets bigger (since you can now target Argon, Boron, Goner), the options of least enemies gets fewer since Goner exploit is not used. However, the toatal workload and efficiency gets a little higher, also because we don't have to suffer from the side effect of Goner exploit any more. After all, since the Goner exploit does not lower the number of enemies but adds difficulty, we will not consider it unless there's a specific combo you want achieve as listed above.

3 enemies option are listed below.

In [22]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    if len(list(least_enemy_set)) == 3:
        workload = sum(get_X(list(set(all_races) - least_enemy_set)))
        efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list += [item]

Terran - Paranid - Yaki, 82.5459352322271, 0.14537360278541045
Terran - Pirates - Yaki, 85.82881902339327, 0.1398131785633601


And with 4 enemies, difficulty lowers significantly.

In [23]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    if len(list(least_enemy_set)) == 4:
        workload = sum(get_X(list(set(all_races) - least_enemy_set)))
        efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list += [item]

Arteus - Paranid - Terran - Yaki, 35.236630370332534, 0.3121751394611627
Arteus - Pirates - Terran - Yaki, 51.070866881961976, 0.2153869842355301
Terran - OTAS - Duke's - Yaki, 74.95096743102711, 0.14676261530743606
Terran - Paranid - OTAS - Yaki, 45.85731953596932, 0.23987446521752934
Terran - OTAS - Pirates - Yaki, 55.976158239538634, 0.19651223567233264
Terran - Paranid - Duke's - Yaki, 62.50956326037024, 0.1759730739788062
Terran - Pirates - Duke's - Yaki, 29.970950403601996, 0.3670220614251188
Terran - Paranid - Pirates - Yaki, 29.044273921030808, 0.3787321394195693
Split - Paranid - Terran - Yaki, 50.401375412268266, 0.21824801228187268
Split - Pirates - Terran - Yaki, 53.26729548048723, 0.20650569736602262
Terran - Paranid - Boron - Yaki, 45.732148368752426, 0.24053101357285045
Pirates - Boron - Terran - Yaki, 48.89364099591666, 0.22497813163308214
Terran - OTAS - Boron - Yaki, 88.81007384601443, 0.12385982269390548
Terran - Paranid - Argon - Yaki, 67.54098818275885, 0.162864066

## Conclusion

- A simple tool is developed
- Goner approach is tested and comprared with normal approach
- All possible combos are given with total workload and efficiency
- Arteus - OTAS - Yaki might be the best solution if want to be friend with Terran, at a great cost of 535.0506351864673, efficiency of 0.022427783859779837